# Data Preprocessing

- resampling & normalization
- data augmentation (noise addition, pitch shift, time stretching)

In [56]:
from utility_data import *

In [57]:
audio_params = {'sample_rate': 32000, 'n_fft': 1024, 'hop_length': 501, 'n_mfcc': 128, 'n_mels': 128, 'feature_size': 2048}

dataset = AudioDataset(
    datafolder="data",
    metadata_csv="train.csv",
    audio_dir="train_audio",
    feature_mode='mel',
    audio_params=audio_params,
    metadata=True
)

AudioDataset can produce a new dataset '.csv' and '/train' with the correct, preprocessed data, which can be reused by loading the data directly to the gpu, without reprocessing every time, which is a major bottleneck to training.

In [ ]:
# adjust as fit, to export more data
# note that the new files will have the same size
# as the original data.
dataset.data = dataset.data.head(1000)
dataset.preprocess(output='train_proc')

In [ ]:
audio_params = {'sample_rate': 32000, 'n_fft': 1024, 'hop_length': 501, 'n_mfcc': 128, 'n_mels': 128, 'feature_size': 2048}

dataset = AudioDataset(
    datafolder="data",
    metadata_csv="train_proc.csv",
    audio_dir="train_proc",
    feature_mode='mel',
    audio_params=audio_params,
    metadata=False
)


In [ ]:
# quantifying the loss of cutting <2.5 sec leftover audios
# in the process of forming the recording 5 sec clips
(dataset.data['duration'] % 5).hist(bins=200)